# Tutorial (4): Post-VQE treatments

Here we show some examples of post-VQE treatments.

[(0) Prepare and run UCCD](#(0)-Prepare-and-run-UCCD)  
[(1) VQD for excited state calculation](#(1)-VQD-for-excited-state-calculation)  
[(2) Nuclear gradients and geometry optimization](#(2)-Nuclear-gradients-and-geometry-optimization)  
[(3) Orbital optimization](#(3)-Orbital-optimization)  


In [1]:
# import necessary modules
from quket import *
from quket.utils import *
import quket.config as cf

mpi4py is not imported. no MPI.


## (0) Prepare and run UCCSD
For this tutorial, first run SAUCCD-VQE

In [2]:
### Create QuketData for H2O ###
Q = create(basis="cc-pvdz", 
           ansatz="sauccgd", 
           n_orbitals =5, 
           n_electrons=6, 
           geometry = "O             ;\
                       H 1 1         ;\
                       H 1 1 2 110   ",
           taper_off = True,
           nroots = 2,
           qubitfci = True
        )
# Run UCCD
Q.run()

Basis set = cc-pvdz

*** Geometry ******************************
  O     0.0000000    0.0000000    0.0000000
  H     1.0000000    0.0000000    0.0000000
  H    -0.3420201    0.0000000    0.9396926
*******************************************

Symmetry C2v : C2v(Abelian)
E[FCI]    = -76.024713570960     (Spin = 1   Ms = 0)
            -75.622866608529     (Spin = 1   Ms = 0)
E[HF]     = -76.020139776626     (Spin = 1   Ms = 0)


Overwritten attributes  contract_2e  of <class 'pyscf.fci.direct_spin1_symm.FCISolver'>


Tapering-Off Results:
List of redundant qubits:  [0, 1, 2, 4]
Qubit: 0    Tau: 1.0 [Z0 Z3 Z5 Z7 Z8]
Qubit: 1    Tau: 1.0 [Z1 Z3 Z5 Z7 Z9]
Qubit: 2    Tau: 1.0 [Z2 Z3 Z6 Z7]
Qubit: 4    Tau: 1.0 [Z4 Z5]

States     transformed.
Operators  transformed.
pauli_list transformed.
NBasis = 24
Davidson convergence achieved.
FCI in Qubits (tapered-off mapping)
(FCI state : E = -76.0247135710  multiplicity = 1.0)
   Basis           Coef
| 000011 > : -0.9982 +0.0000i

Entered VQE driver
Performing VQE for sauccgd
Number of VQE parameters: 38
Initial configuration: |111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccgd] = -76.020139776627  <S**2> = +0.000000  Fidelity = 0.996335  rho = 1  
      1: E[sauccgd] = -76.022752097784  <S**2> = +0.000000  Fidelity = 0.998336  Grad = 1.29e-01  CPU Time =    0.24836  (0.00 / step)
      2: E[sauccgd] = -76.024032215438  <S**2> = +0.000000  Fidelity = 0.999502  Grad = 8.93e-02 

## (1) VQD for excited state calculation
`vqd()` prepares for excited state calculation with variational quantum deflation (VQD).  
The previous VQE has to be converged: see
```python
   Q.converge
```
and the penalty parameter is set to the previous energy, `Q.energy`.  
It is also advisable to use an initial state that is approximately orthogonal to the previous states.

#### First copy `Q` to a new `QuketData` instance.

In [3]:
VQD = Q.copy()

### Point-group symmetry
Due to the point-group symmetry C$2v$ of H$_2$O, the ground state possesses an A$_g$ symmetry.   
The first singlet excited state we aim for should have the same symmetry as the ground state.  
Otherwise, VQD makes no sense especially with qubits being tapered-off by point-group symmetry.  
To ensure the point-group symmetry of the VQD state, it is convenient to look at the MO symmetries.

In [4]:
VQD.print_mo_energy()


[Molecular Orbitals]
---+-------+------------+------------------
 # |  Sym  |   energy   |     category
---+-------+------------+------------------
 0 |   A1  |   -20.5544 | Frozen Core
 1 |   A1  |    -1.3121 | Frozen Core
 2 |   B1  |    -0.6862 | Active (occupied)
 3 |   A1  |    -0.5477 | Active (occupied)
 4 |   B2  |    -0.4881 | Active (occupied)
 5 |   A1  |     0.1785 | Active (virtual)
 6 |   B1  |     0.2493 | Active (virtual)
 7 |   B1  |     0.7792 | Secondary
 8 |   A1  |     0.7921 | Secondary
 9 |   A1  |     1.1644 | Secondary
10 |   B2  |     1.2004 | Secondary
11 |   B1  |     1.2501 | Secondary
12 |   A2  |     1.4506 | Secondary
13 |   A1  |     1.4981 | Secondary
14 |   B2  |     1.6943 | Secondary
15 |   B1  |     1.8621 | Secondary
16 |   A1  |     1.8889 | Secondary
17 |   B1  |     2.3708 | Secondary
18 |   A1  |     2.3817 | Secondary
19 |   B2  |     3.2223 | Secondary
20 |   A2  |     3.3175 | Secondary
21 |   A1  |     3.4172 | Secondary
22 |   A1  |     

#### VQD setting with `vqd()`
Based on the orbital symmetries above, the lowest excited state of $A_g$ is orbital `3` -> orbital `5`.
To create a singlet state, a two-determinant initial state 
\begin{equation}
\frac{1}{\sqrt{2}} \left( |0001111011\rangle - |0010110111\rangle\right)
\end{equation}
should be prepared.

In [5]:
VQD.vqd(det="1 * |0001111011> - 1 * |0010110111>")

Performing VQD for excited state 1
Tapered-off initial state...
pauli_list transformed.
Tapered-off pauli list...
VQD ready. Perform run().


#### Check the state.
Note that because of `taper_off = True`, the initial state for VQE is also tapered-off.

In [6]:
print_state(VQD.init_state)

   Basis           Coef
| 000111 > : +0.7071 +0.0000i
| 001010 > : -0.7071 +0.0000i



#### Backtransform and check the state.
A tapered state can be backtransformed to the original mapping by `transform_state()`.

In [7]:
test_state, success = VQD.transform_state(VQD.init_state, backtransform=True)
print("Transformation success?", success)
print_state(test_state)

Transformation success? True
     Basis             Coef
| 0001111011 > : +0.7071 +0.0000i
| 0010110111 > : -0.7071 +0.0000i



#### This is indeed $\frac{1}{\sqrt{2}} \left( |0001111011\rangle - |0010110111\rangle\right)$. 
### Now run VQD.

In [8]:
VQD.run()

Entered VQE driver
Performing VQE for sauccgd
Number of VQE parameters: 38
Initial configuration:  +1.0000+0.0000j * |1111011> -1.0000+0.0000j * |10110111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytical
Circuit order: Exp[T1] Exp[T2] |0>
Initial: E[sauccgd] = -75.593113704590  <S**2> = -0.000000  Fidelity = 0.000176  rho = 1  
      1: E[sauccgd] = -75.591476707646  <S**2> = +0.000000  Fidelity = 0.000001  Grad = 1.07e+00  CPU Time =    0.33908  (0.00 / step)
      2: E[sauccgd] = -75.597543526677  <S**2> = +0.000000  Fidelity = 0.000004  Grad = 2.20e+00  CPU Time =    0.26645  (0.00 / step)
      3: E[sauccgd] = -75.610505662102  <S**2> = +0.000000  Fidelity = 0.000019  Grad = 3.72e+00  CPU Time =    0.09587  (0.00 / step)
      4: E[sauccgd] = -75.612692928148  <S**2> = +0.000000  Fidelity = 0.000002  Grad = 1.58e+00  CPU Time =    0.09749  (0.00 / step)
      5: E[sauccgd] = -75.613407236309  <S**2> = +0.000000  Fidelity = 0.000001  Grad = 3.88e-01  CPU Tim

#### Note that VQD, especially with generalized UCC, is a highly nonlinear problem and may take a long time to converge.
### Check the energy

In [9]:
print("FCI energies:        ", VQD.fci_energy)
print("SA-UCCGD VQE energy:  ", VQD.lower_states[0]['energy'])
print("SA-UCCGD VQD energy:  ", VQD.energy)

FCI energies:         [-76.02471357095992 -75.62286660852946]
SA-UCCGD VQE energy:   -76.02470887824937
SA-UCCGD VQD energy:   -75.62284443523278


## (2) Nuclear gradients and geometry optimization
Nuclear gradient calculation and geometry optimization can be run with `grad()` and `opt()`.  
If qubits are tapered-off, states should be backtransformed because computing reduced density matrices in the reduced mapping is currently not supported.

In [10]:
Grad = Q.copy()
# Compute nuclear gradients
Grad.taper_off(backtransform=True)
Grad.grad()

States     backtransformed.
Operators  backtransformed.
pauli_list backtransformed.
theta_list backtransformed.
 === Computing 1RDM === 
 === Computing 2RDM === 

*** Nuclear Gradients *********************
            x            y            z
  O    -0.0177772    0.0000000   -0.0253885
  H     0.0450532   -0.0000000   -0.0126284
  H    -0.0272759   -0.0000000    0.0380170
*******************************************



### Geometry optimization requires the installation of `pyberny`

In [11]:
OPT = Q.copy()
# Geometry optimization
OPT.taper_off()
OPT.opt()

Current states are already tapered-off. No transformation done
Current operators are already tapered-off. No transformation done
Current pauli_list is already tapered-off. No transformation done
Current theta_list is already tapered-off. No transformation done
States     backtransformed.
Operators  backtransformed.
pauli_list backtransformed.
theta_list backtransformed.
 === Computing 1RDM === 
 === Computing 2RDM === 

*** Nuclear Gradients *********************
            x            y            z
  O    -0.0177772    0.0000000   -0.0253885
  H     0.0450532   -0.0000000   -0.0126284
  H    -0.0272759   -0.0000000    0.0380170
*******************************************

"""""""""""""""""""
     Cycle 0
"""""""""""""""""""
Basis set = cc-pvdz

*** Geometry ******************************
  O    -0.0076740    0.0000000   -0.0109596
  H     0.9435717    0.0000000    0.0476781
  H    -0.2779178    0.0000000    0.9029742
*******************************************

Symmetry C2v : C2v(A

## (3) Orbital optimization 
`oo()` performs orbital optimization.  
It assumes the method is VQE, but works for most ansatze.

In [12]:
# This example provides oo-SAUCCD
OO = Q.copy()
OO.oo()

States     backtransformed.
Operators  backtransformed.
pauli_list backtransformed.
theta_list backtransformed.


""""""""""""""""""""""""""""""""""""""
     Orbital-Optimization Cycle 1
""""""""""""""""""""""""""""""""""""""

 === Computing 1RDM === 
 === Computing 2RDM === 
Running Davidson to get update by AH
Target lowest eigenvalue of augmented Hessian:  -0.032562491087009524  (C0 = 0.5420956622835309)
Golden Section done in 2



||g|| = 0.03480523170083466 ---> Updated with scaling factor 0.8594235253127366


Next energy estimation:  -76.05014212152284
Davidson convergence achieved.
FCI in Qubits
(FCI state : E = -76.0693695570  multiplicity = 1.0)
     Basis             Coef
| 0000111111 > : +0.9895 +0.0000i

States     transformed.
Operators  transformed.
pauli_list transformed.
theta_list transformed.
Entered VQE driver
Performing VQE for sauccgd
Number of VQE parameters: 38
Initial configuration: |111111>
Convergence criteria: ftol = 1E-09, gtol = 1E-05
Derivatives: Analytica